# Vision-Language Model (VLM) OCR Benchmarking

Testing **Vision-Language Models** for OCR on historical SOCAR documents.

## Why VLMs for OCR?
- **Better than traditional OCR** (Tesseract, EasyOCR, etc.)
- **Understands context** - can handle handwriting, layout, multi-language
- **Directly processes images** - no separate OCR step needed
- **State-of-the-art accuracy** on complex documents

## Vision Models to Test:
1. **GPT-4.1** ⭐⭐⭐⭐⭐ (Excellent OCR capability)
2. **GPT-5, GPT-5-mini** ⭐⭐⭐⭐⭐ (Latest, best performance)
3. **Claude-Sonnet-4.5** ⭐⭐⭐⭐⭐ (Very good OCR)
4. **Phi-4-multimodal-instruct** ⭐⭐⭐⭐ (Explicitly multimodal)
5. **Llama-4-Maverick-17B** ⭐⭐⭐⭐ (May have vision support)
6. **DeepSeek-VL** (Vision-Language specialized)

## Metrics:
- **CER** (Character Error Rate) - Lower is better
- **CSR** (Character Success Rate) = 100 - CER - Higher is better
- **WER** (Word Error Rate) - Lower is better
- **WSR** (Word Success Rate) = 100 - WER - Higher is better

In [1]:
# Install required packages
# !pip install openai PyMuPDF Pillow jiwer pandas matplotlib seaborn python-dotenv

In [2]:
import os
import base64
import re
import time
from pathlib import Path
from typing import Dict, List, Tuple
from io import BytesIO

from dotenv import load_dotenv
import fitz  # PyMuPDF
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from jiwer import wer, cer
from openai import AzureOpenAI

# Load environment
load_dotenv()

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("✅ Libraries loaded")

✅ Libraries loaded


In [3]:
# Auto-detect project root (works from any directory)
import os
from pathlib import Path

if Path('data').exists() and Path('docs').exists():
    # Already in project root
    PROJECT_ROOT = Path.cwd()
elif Path('../data').exists() and Path('../docs').exists():
    # In notebooks/ subdirectory
    PROJECT_ROOT = Path.cwd().parent
else:
    # Fallback: try to find project root
    current = Path.cwd()
    while current != current.parent:
        if (current / 'data').exists() and (current / 'docs').exists():
            PROJECT_ROOT = current
            break
        current = current.parent
    else:
        PROJECT_ROOT = Path.cwd()

# Define all paths relative to project root
DATA_DIR = PROJECT_ROOT / 'data'
DOCS_DIR = PROJECT_ROOT / 'docs'
OUTPUT_DIR = PROJECT_ROOT / 'output'
PDFS_DIR = DATA_DIR / 'pdfs'

print(f"✅ Project root: {PROJECT_ROOT}")
print(f"✅ Data directory: {DATA_DIR}")
print(f"✅ PDFs directory: {PDFS_DIR}")
print(f"✅ Output directory: {OUTPUT_DIR}")

✅ Project root: /Users/ismatsamadov/SOCAR_Hackathon
✅ Data directory: /Users/ismatsamadov/SOCAR_Hackathon/data
✅ PDFs directory: /Users/ismatsamadov/SOCAR_Hackathon/data/pdfs
✅ Output directory: /Users/ismatsamadov/SOCAR_Hackathon/output


In [4]:
def load_ground_truth(md_path: str) -> str:
    """
    Load ground truth text from markdown file.
    Removes markdown formatting for pure text comparison.
    """
    with open(md_path, 'r', encoding='utf-8') as f:
        text = f.read()
    
    # Remove markdown elements
    text = re.sub(r'^#+\s+', '', text, flags=re.MULTILINE)  # Headers
    text = re.sub(r'\*\*(.+?)\*\*', r'\1', text)  # Bold
    text = re.sub(r'\*(.+?)\*', r'\1', text)  # Italic
    text = re.sub(r'---+', '', text)  # Horizontal rules
    text = re.sub(r'\n\s*\n+', '\n\n', text)  # Normalize newlines
    
    return text.strip()

# Load ground truth - using dynamic path
ground_truth = load_ground_truth(str(DATA_DIR / 'document_00.md'))
print(f"✅ Ground truth loaded: {len(ground_truth)} characters")
print(f"Preview:\n{ground_truth[:300]}...")

✅ Ground truth loaded: 22386 characters
Preview:
XÜLASƏ

Bu tədqiqat Aşağı Kür çökəkliyi (AKÇ) və Bakı arxipelaqı (BA) daxil olmaqla Cənubi Xəzər çökəkliyi sistemində faydalı qazıntıların mənşəyinin paleotektonik, paleocoğrafi şərait və geodinamik rejimlə necə əlaqələndiyini, eləcə də Gec Miosendən etibarən Ərəbistan plitəsinin təsiri ilə formalaş...


In [5]:
def pdf_to_images(pdf_path: str, dpi: int = 150) -> List[Image.Image]:
    """
    Convert PDF pages to PIL Images.
    Higher DPI = better quality for VLMs.
    """
    doc = fitz.open(pdf_path)
    images = []
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        # Render at higher resolution
        zoom = dpi / 72  # 72 DPI is default
        mat = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=mat)
        
        # Convert to PIL Image
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        images.append(img)
    
    doc.close()
    return images

def image_to_base64(image: Image.Image, format: str = 'PNG') -> str:
    """
    Convert PIL Image to base64 string for API.
    """
    buffered = BytesIO()
    image.save(buffered, format=format)
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

# Test conversion - using dynamic path
pdf_path = str(PDFS_DIR / 'document_00.pdf')
test_images = pdf_to_images(pdf_path)
print(f"\n✅ Converted PDF to {len(test_images)} images")
print(f"First image size: {test_images[0].size}")


✅ Converted PDF to 12 images
First image size: (3072, 4096)


In [7]:
# Initialize Azure OpenAI client
azure_client = AzureOpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION', '2024-08-01-preview'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT')
)

# Vision model configurations
VLM_MODELS = {
    'GPT-4.1': {
        'deployment': 'gpt-4.1',
        'supports_vision': True,
        'rating': '⭐⭐⭐⭐⭐',
        'notes': 'Excellent OCR'
    },
    'GPT-5': {
        'deployment': 'gpt-5',
        'supports_vision': True,
        'rating': '⭐⭐⭐⭐⭐',
        'notes': 'Latest model'
    },
    'GPT-5-mini': {
        'deployment': 'gpt-5-mini',
        'supports_vision': True,
        'rating': '⭐⭐⭐⭐⭐',
        'notes': 'Fast + excellent'
    },
    'Claude-Sonnet-4.5': {
        'deployment': 'claude-sonnet-4-5',
        'supports_vision': True,
        'rating': '⭐⭐⭐⭐⭐',
        'notes': 'Very good OCR'
    },
    'Phi-4-multimodal': {
        'deployment': 'Phi-4-multimodal-instruct',
        'supports_vision': True,
        'rating': '⭐⭐⭐⭐',
        'notes': 'Explicitly multimodal'
    },
    'Llama-4-Maverick-17B': {
        'deployment': 'Llama-4-Maverick-17B-128E-Instruct-FP8',
        'supports_vision': True,
        'rating': '⭐⭐⭐⭐',
        'notes': 'Testing vision capability'
    }
}

print(f"✅ Configured {len(VLM_MODELS)} vision models")

✅ Configured 6 vision models


In [8]:
def vlm_extract_text(model_name: str, images: List[Image.Image], 
                     temperature: float = 0.0) -> Tuple[str, float]:
    """
    Extract text from images using Vision-Language Model.
    Returns: (extracted_text, response_time)
    """
    deployment = VLM_MODELS[model_name]['deployment']
    
    # OCR prompt - optimized for accuracy
    system_prompt = """You are an expert OCR system for historical oil & gas documents.

Extract ALL text from the image with 100% accuracy. Follow these rules:
1. Preserve EXACT spelling - including Azerbaijani, Russian, and English text
2. Maintain original Cyrillic characters - DO NOT transliterate
3. Keep all numbers, symbols, and special characters exactly as shown
4. Preserve layout structure (paragraphs, line breaks)
5. Include ALL text - headers, body, footnotes, tables, captions
6. If text is unclear, make best effort but stay accurate

Output ONLY the extracted text. No explanations, no descriptions."""
    
    try:
        all_text = []
        total_time = 0
        
        for page_num, image in enumerate(images, 1):
            # Convert image to base64
            image_base64 = image_to_base64(image)
            
            # Prepare messages with image
            messages = [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": f"Extract all text from page {page_num}:"
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{image_base64}"
                            }
                        }
                    ]
                }
            ]
            
            # Call VLM with appropriate token parameter
            start_time = time.time()
            
            # GPT-5 models use max_completion_tokens, others use max_tokens
            if deployment.startswith('gpt-5'):
                response = azure_client.chat.completions.create(
                    model=deployment,
                    messages=messages,
                    temperature=temperature,
                    max_completion_tokens=4000
                )
            else:
                response = azure_client.chat.completions.create(
                    model=deployment,
                    messages=messages,
                    temperature=temperature,
                    max_tokens=4000
                )
            
            elapsed = time.time() - start_time
            total_time += elapsed
            
            # Extract text
            page_text = response.choices[0].message.content
            all_text.append(page_text)
            
            print(f"  Page {page_num}/{len(images)}: {elapsed:.1f}s")
        
        # Combine all pages
        full_text = '\n\n'.join(all_text)
        return full_text, total_time
    
    except Exception as e:
        return f"ERROR: {str(e)}", 0.0

print("✅ VLM extraction function ready")

✅ VLM extraction function ready


## 4. Metrics Calculation

## 5. Run Benchmark on All VLMs

In [10]:
# Select models to test
MODELS_TO_TEST = [
    'GPT-4.1',
    'GPT-5',
    'GPT-5-mini',
    'Claude-Sonnet-4.5',
    'Phi-4-multimodal',
    'Llama-4-Maverick-17B',  # Added for comparison
]

print(f"Testing {len(MODELS_TO_TEST)} vision models...")
print("This will take several minutes...\n")

Testing 6 vision models...
This will take several minutes...



In [11]:
# Prepare PDF images
images = pdf_to_images(pdf_path, dpi=150)
print(f"Prepared {len(images)} page images\n")

# Run benchmark
results = []

for model_name in MODELS_TO_TEST:
    print(f"\n{'='*80}")
    print(f"Testing: {model_name} {VLM_MODELS[model_name]['rating']}")
    print(f"Notes: {VLM_MODELS[model_name]['notes']}")
    print(f"{'='*80}")
    
    # Extract text
    extracted_text, response_time = vlm_extract_text(model_name, images)
    
    if extracted_text.startswith('ERROR'):
        print(f"❌ Failed: {extracted_text}")
        continue
    
    print(f"\n✅ Total time: {response_time:.2f}s")
    print(f"✅ Extracted: {len(extracted_text)} characters")
    
    # Calculate metrics
    metrics = calculate_ocr_metrics(ground_truth, extracted_text)
    
    # Store result
    result = {
        'Model': model_name,
        'Response_Time': round(response_time, 2),
        **metrics,
        'Rating': VLM_MODELS[model_name]['rating'],
        'Notes': VLM_MODELS[model_name]['notes'],
        'Extracted_Preview': extracted_text[:200]
    }
    
    results.append(result)
    
    # Show summary
    print(f"\n📊 Metrics:")
    print(f"   CSR (Character Success): {metrics['CSR']:.2f}%")
    print(f"   WSR (Word Success): {metrics['WSR']:.2f}%")
    print(f"   CER (Character Error): {metrics['CER']:.2f}%")
    print(f"   WER (Word Error): {metrics['WER']:.2f}%")

print(f"\n{'='*80}")
print("✅ VLM OCR Benchmarking complete!")
print(f"{'='*80}")

Prepared 12 page images


Testing: GPT-4.1 ⭐⭐⭐⭐⭐
Notes: Excellent OCR
  Page 1/12: 9.2s
  Page 2/12: 8.8s
  Page 3/12: 9.7s
  Page 4/12: 10.7s
  Page 5/12: 12.4s
  Page 6/12: 9.9s
  Page 7/12: 12.1s
  Page 8/12: 10.7s
  Page 9/12: 12.0s
  Page 10/12: 10.9s
  Page 11/12: 7.7s
  Page 12/12: 6.6s

✅ Total time: 120.73s
✅ Extracted: 22018 characters

📊 Metrics:
   CSR (Character Success): 86.17%
   WSR (Word Success): 68.19%
   CER (Character Error): 13.83%
   WER (Word Error): 31.81%

Testing: GPT-5 ⭐⭐⭐⭐⭐
Notes: Latest model
❌ Failed: ERROR: Completions.create() got an unexpected keyword argument 'max_completion_tokens'

Testing: GPT-5-mini ⭐⭐⭐⭐⭐
Notes: Fast + excellent
❌ Failed: ERROR: Completions.create() got an unexpected keyword argument 'max_completion_tokens'

Testing: Claude-Sonnet-4.5 ⭐⭐⭐⭐⭐
Notes: Very good OCR
❌ Failed: ERROR: Error code: 400 - {'error': {'code': 'unknown_model', 'message': 'Unknown model: claude-sonnet-4-5', 'details': 'Unknown model: claude-sonnet-4-5'}}

Testi

## 6. Results Analysis

In [12]:
# Create DataFrame
df = pd.DataFrame(results)

# Sort by CSR (best first)
df = df.sort_values('CSR', ascending=False).reset_index(drop=True)

# Display results
display_cols = ['Model', 'CSR', 'WSR', 'CER', 'WER', 'Response_Time', 'Rating']

print("\n" + "="*100)
print("📊 VLM OCR BENCHMARKING RESULTS")
print("="*100)
print(df[display_cols].to_string(index=False))
print("="*100)


📊 VLM OCR BENCHMARKING RESULTS
  Model   CSR   WSR   CER   WER  Response_Time Rating
GPT-4.1 86.17 68.19 13.83 31.81         120.73  ⭐⭐⭐⭐⭐


In [ ]:
# Create comprehensive visualization
import os
from pathlib import Path

# Create output directory - using dynamic path
output_dir = OUTPUT_DIR / 'vlm_ocr_benchmark'
output_dir.mkdir(parents=True, exist_ok=True)

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

models = df['Model'].tolist()
colors = sns.color_palette('viridis', len(models))

# 1. CSR - Character Success Rate (MAIN METRIC)
ax1 = axes[0, 0]
bars1 = ax1.barh(models, df['CSR'], color=colors)
ax1.set_xlabel('CSR (%) - Higher is Better', fontsize=12, fontweight='bold')
ax1.set_title('Character Success Rate (CSR)\n🏆 HACKATHON PRIMARY METRIC', 
              fontsize=14, fontweight='bold')
ax1.set_xlim(0, 100)
for i, (model, csr) in enumerate(zip(models, df['CSR'])):
    ax1.text(csr + 1, i, f'{csr:.2f}%', va='center', fontsize=11, fontweight='bold')
ax1.axvline(x=90, color='green', linestyle='--', alpha=0.3, label='Excellent (>90%)')
ax1.axvline(x=80, color='orange', linestyle='--', alpha=0.3, label='Good (>80%)')
ax1.legend(fontsize=9)

# 2. WSR - Word Success Rate
ax2 = axes[0, 1]
bars2 = ax2.barh(models, df['WSR'], color=colors)
ax2.set_xlabel('WSR (%) - Higher is Better', fontsize=12, fontweight='bold')
ax2.set_title('Word Success Rate (WSR)', fontsize=14, fontweight='bold')
ax2.set_xlim(0, 100)
for i, (model, wsr) in enumerate(zip(models, df['WSR'])):
    ax2.text(wsr + 1, i, f'{wsr:.2f}%', va='center', fontsize=11, fontweight='bold')

# 3. Response Time
ax3 = axes[1, 0]
bars3 = ax3.barh(models, df['Response_Time'], color=colors)
ax3.set_xlabel('Total Time (seconds) - Lower is Better', fontsize=12, fontweight='bold')
ax3.set_title('Processing Speed', fontsize=14, fontweight='bold')
for i, (model, time_val) in enumerate(zip(models, df['Response_Time'])):
    ax3.text(time_val + 0.5, i, f'{time_val:.1f}s', va='center', fontsize=11)

# 4. Error Rates Comparison
ax4 = axes[1, 1]
x = range(len(models))
width = 0.35
ax4.bar([i - width/2 for i in x], df['CER'], width, label='CER', color='coral', alpha=0.8)
ax4.bar([i + width/2 for i in x], df['WER'], width, label='WER', color='skyblue', alpha=0.8)
ax4.set_ylabel('Error Rate (%) - Lower is Better', fontsize=12, fontweight='bold')
ax4.set_title('Error Rates', fontsize=14, fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels(models, rotation=45, ha='right')
ax4.legend(fontsize=11)
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'results.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Visualization saved to '{output_dir}/results.png'")

In [ ]:
# Save results
from pathlib import Path

# Using dynamic path
output_dir = OUTPUT_DIR / 'vlm_ocr_benchmark'
output_dir.mkdir(parents=True, exist_ok=True)

df.to_csv(output_dir / 'detailed_results.csv', index=False, encoding='utf-8')
rankings.to_csv(output_dir / 'rankings.csv', index=False, encoding='utf-8')

print("\n✅ Results exported to output/vlm_ocr_benchmark/:")
print("   - detailed_results.csv")
print("   - rankings.csv")
print("   - results.png")

## 10. Sample Text Comparison